# Sahithi Paruchuri

# Character-level translation

In [ ]:
# Import the packages
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [13]:
#Configuring the model
batch_size = 32 #The training batch size
epochs = 100 #Many epochs better during final training
num_samples = 10000 #The sample size we will train on
data = "por.txt" #The data location in file system

In [14]:
dataset = pd.read_table("por.txt")
dataset.head(10)

,Go.,Vai.,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1196331 (alexmarcelo)
0,Go.,Vá.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,Oi.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,Corre!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,Corra!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,Corram!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
5,Run.,Corre!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
6,Run.,Corra!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
7,Run.,Corram!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
8,Who?,Quem?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
9,Who?,Que,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [15]:
#Data Cleaning and preparation
#Vectorizing the dataset
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data, encoding='utf-8').read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text,_ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#Creating the dictionary for the characters
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

#Creating the encoder input data
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float64')

#Creating the decoder input data
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float64')

#Creating the decoder target data
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float64')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 86
Max sequence length for inputs: 15
Max sequence length for outputs: 45


In [16]:
#Building the model so that we can test it
#Building the encoder
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#Building the decoder
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                        initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Defining the model that will turn
#`encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

2022-11-19 15:37:05.433694: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
#Compiling the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2)

#Saving the model
model.save('s2s')

Epoch 1/100
250/250 [==============================] - 26s 96ms/step - loss: 1.1597 - accuracy: 0.7071 - val_loss: 1.0182 - val_accuracy: 0.7272
Epoch 2/100
250/250 [==============================] - 24s 97ms/step - loss: 0.7631 - accuracy: 0.7826 - val_loss: 0.8183 - val_accuracy: 0.7586
Epoch 3/100
250/250 [==============================] - 25s 98ms/step - loss: 0.6408 - accuracy: 0.8094 - val_loss: 0.7458 - val_accuracy: 0.7766
Epoch 4/100
250/250 [==============================] - 24s 98ms/step - loss: 0.5735 - accuracy: 0.8291 - val_loss: 0.6922 - val_accuracy: 0.7926
Epoch 5/100
250/250 [==============================] - 24s 95ms/step - loss: 0.5250 - accuracy: 0.8432 - val_loss: 0.6485 - val_accuracy: 0.8056
Epoch 6/100
250/250 [==============================] - 24s 95ms/step - loss: 0.4876 - accuracy: 0.8545 - val_loss: 0.6148 - val_accuracy: 0.8188
Epoch 7/100
250/250 [==============================] - 24s 96ms/step - loss: 0.4563 - accuracy: 0.8632 - val_loss: 0.5982 - val_ac

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [18]:
#Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model('s2s')

#Building the encoder model
encoder_model = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_model, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(256,))
decoder_state_input_c = keras.Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)



#Reverse-lookup token index to decode sequences back to
#something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == "\n" or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    
    return decoded_sentence


In [19]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Go.
Decoded sentence: Vá.

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Go.
Decoded sentence: Vá.

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Hi.
Decoded sentence: Oi.

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Run!
Decoded sentence: Corra!

1/1 [==============================] - 0s 13ms/step
-
Input sentence: Run!
Decoded sentence: Corra!

1/1 [==============================] - 0s 11ms/step
-
Input sentence: Run!
Decoded sentence: Corra!

1/1 [==============================] - 0s 13ms/step
-
Input sentence: Run.
Decoded sentence: Corra!

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Run.
Decoded sentence: Corra!

1/1 [==============================] - 0s 13ms/step
-
Input sentence: Run.
Decoded sentence: Corra!

1/1 [==============================] - 0s 11ms/step
-
Input sentence: Who?
Decoded sentence: Que

